![Dark Souls Menu Theme](notes/DarkSoulsMainMenuTheme.png)


In [ ]:
# -----------------------------
# Define melody based on the sheet (first part simplified)
# -----------------------------
def get_dark_souls_theme_notes():
    return [
        # Bar 1
        ("E4", 0.125), ("G4", 0.125), ("A4", 0.125), ("B4", 0.125),
        ("A4", 0.125), ("G4", 0.125), ("E4", 0.125), ("G4", 0.125),
        # Bar 2
        ("F4", 0.125), ("A4", 0.125), ("B4", 0.125), ("C5", 0.125),
        ("B4", 0.125), ("A4", 0.125), ("F4", 0.125), ("A4", 0.125),
        # Bar 3
        ("E4", 0.125), ("G4", 0.125), ("A4", 0.125), ("B4", 0.125),
        ("A4", 0.125), ("G4", 0.125), ("E4", 0.125), ("G4", 0.125),
        # Bar 4
        ("F4", 0.125), ("A4", 0.125), ("B4", 0.125), ("C5", 0.125),
        ("B4", 0.125), ("A4", 0.125), ("F4", 0.125), ("A4", 0.125),
        # Bar 5
        ("E4", 0.125), ("G4", 0.125), ("A4", 0.125), ("B4", 0.125),
        ("A4", 0.125), ("G4", 0.125), ("E4", 0.125), ("G4", 0.125),
        # Bar 6
        ("F4", 0.125), ("A4", 0.125), ("B4", 0.125), ("C5", 0.125),
        ("B4", 0.125), ("A4", 0.125), ("F4", 0.125), ("A4", 0.125),
        # Bar 7
        ("E4", 0.125), ("G4", 0.125), ("A4", 0.125), ("B4", 0.125),
        ("A4", 0.125), ("G4", 0.125), ("E4", 0.125), ("G4", 0.125),
        # Bar 8
        ("F4", 0.125), ("A4", 0.125), ("B4", 0.125), ("C5", 0.125),
        ("B4", 0.125), ("A4", 0.125), ("F4", 0.125), ("A4", 0.125),
    ]


In [ ]:
# Dark Souls Menu Theme Synthesizer
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import scipy.signal as signal

# -----------------------------
# Generate 12-tone chromatic scale (C4–B4)
# -----------------------------
def generate_multi_octave_scale_guitar(fs=16000, duration=1.0, octave_range=3):
    note_names = [
        ("C", 0), ("C#", 1), ("Db", 1),
        ("D", 2), ("D#", 3), ("Eb", 3),
        ("E", 4),
        ("F", 5), ("F#", 6), ("Gb", 6),
        ("G", 7), ("G#", 8), ("Ab", 8),
        ("A", 9), ("A#", 10), ("Bb", 10),
        ("B", 11)
    ]

    tones = {}
    base_freq = 130.81  # C3

    def karplus_strong(freq, duration, fs=16000, decay=0.996):
        N = int(fs / freq)
        noise = np.random.uniform(-1, 1, N)
        output = np.zeros(int(fs * duration))
        output[:N] = noise

        for i in range(N, len(output)):
            output[i] = decay * 0.5 * (output[i - N] + output[i - N - 1])

        return output


    for octave in range(octave_range):  # Octaves 3, 4, 5
        for name, semitone in note_names:
            full_name = f"{name}{3 + octave}"
            freq = base_freq * (2 ** (octave + semitone / 12))
            t = np.linspace(0, duration, int(fs * duration), endpoint=False)
            tone = karplus_strong(freq, duration, fs)
            tones[full_name] = tone

    return tones, fs


def generate_multi_octave_scale_piano(fs=16000, duration=1.0, octave_range=3):
    note_names = [
        ("C", 0), ("C#", 1), ("Db", 1),
        ("D", 2), ("D#", 3), ("Eb", 3),
        ("E", 4),
        ("F", 5), ("F#", 6), ("Gb", 6),
        ("G", 7), ("G#", 8), ("Ab", 8),
        ("A", 9), ("A#", 10), ("Bb", 10),
        ("B", 11)
    ]

    tones = {}
    base_freq = 130.81  # C3

    def piano_karplus_strong(freq, duration, fs=16000, decay=0.996):
        N = int(fs / freq)
        hammer = np.exp(-np.linspace(0, 4, N)) * np.random.uniform(-1, 1, N)
        output = np.zeros(int(fs * duration))
        output[:N] = hammer
        for i in range(N, len(output)):
            output[i] = decay * 0.5 * (output[i - N] + output[i - N - 1])
        return output

    for octave in range(octave_range):  # Octaves 3–5
        for name, semitone in note_names:
            full_name = f"{name}{3 + octave}"
            freq = base_freq * (2 ** (octave + semitone / 12))
            tone = piano_karplus_strong(freq, duration, fs)  # ✅ duration instead of t
            tone *= np.hanning(len(tone))  # Apply window
            tones[full_name] = tone

    return tones, fs



# -----------------------------
# Concatenate melody
# -----------------------------
def synthesize_melody(tones, fs, melody_notes, base_duration=1.0, fade_ratio=0.05):
    """
    Concatenate tones into a melody.
    Each note is faded in/out using a Hann window on the start and end.

    Parameters:
        tones         : dict of note_name -> waveform
        fs            : sampling rate
        melody_notes  : list of (note_name, duration_factor)
        base_duration : length of a "whole" note in seconds
        fade_ratio    : fraction of the note length used for fade in/out (0.05 = 5%)
    """
    melody = []
    for note_name, length_factor in melody_notes:
        tone = tones[note_name]
        total_samples = int(fs * base_duration * length_factor)

        # Trim or pad tone to target length
        if len(tone) > total_samples:
            tone = tone[:total_samples]
        else:
            tone = np.pad(tone, (0, total_samples - len(tone)))

        # Apply fade-in/out using Hann window
        fade_len = int(total_samples * fade_ratio)
        if fade_len > 0:
            fade_in = np.hanning(fade_len * 2)[:fade_len]
            fade_out = np.hanning(fade_len * 2)[fade_len:]
            tone[:fade_len] *= fade_in
            tone[-fade_len:] *= fade_out

        melody.append(tone)

    return np.concatenate(melody)

# rir noise origin: https://www.openslr.org/28/
def apply_rir(signal_in, rir_path, fs):
    rir, rir_fs = sf.read(rir_path)
    if rir_fs != fs:
        raise ValueError(f"RIR sampling rate {rir_fs} must match melody rate {fs}")

    if rir.ndim == 1:
        # Mono RIR
        return signal.fftconvolve(signal_in, rir, mode='full')
    elif rir.ndim == 2:
        # Stereo RIR (left and right)
        left = signal.fftconvolve(signal_in, rir[:, 0], mode='full')
        right = signal.fftconvolve(signal_in, rir[:, 1], mode='full')
        return np.stack([left, right], axis=1)
    else:
        raise ValueError("Unsupported RIR format")



In [ ]:
tones, fs = generate_multi_octave_scale_guitar(fs=16000, duration=1.0)
melody_notes = get_dark_souls_theme_notes()

# You can adjust this to control the song speed
melody = synthesize_melody(tones, fs, melody_notes, base_duration=3.5, fade_ratio=0.1)

# apply reverb
rir_path = "rirs/RIRS_NOISES/real_rirs_isotropic_noises/air_type1_air_binaural_stairway_1_1_90.wav"
melody = apply_rir(melody, rir_path, fs)


# Save to WAV
sf.write("dark_souls_theme_synth.wav", melody, fs)

# Plot waveform
plt.figure(figsize=(10, 3))
plt.plot(melody[:10000])
plt.title("Dark Souls Menu Theme (Guitar Synth)")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Play the sound
import IPython.display as ipd
ipd.Audio("dark_souls_theme_synth.wav")

![Demon Souls](notes/DemonsSouls_SoulOfAMist.png)

In [ ]:
def get_demons_souls_theme_notes():
    return [
        # Bar 1
        ("G4", 0.25), ("Bb4", 0.25), ("D5", 0.5),
        # Bar 2
        ("F4", 0.25), ("G4", 0.25), ("Bb4", 0.5),
        # Bar 3
        ("D5", 0.25), ("C5", 0.25), ("Bb4", 0.5),
        # Bar 4
        ("G4", 0.25), ("F4", 0.25), ("D4", 0.5),
        # Bar 5
        ("G4", 0.25), ("Bb4", 0.25), ("D5", 0.5),
        # Bar 6
        ("F4", 0.25), ("G4", 0.25), ("Bb4", 0.5),
        # Bar 7
        ("D5", 0.25), ("Eb5", 0.25), ("F5", 0.5),
        # Bar 8
        ("G5", 0.25), ("F5", 0.25), ("Eb5", 0.5),

        # Bar 9
        ("D5", 0.25), ("C5", 0.25), ("Bb4", 0.5),
        # Bar 10
        ("F4", 0.25), ("G4", 0.25), ("Bb4", 0.5),
        # Bar 11
        ("D5", 0.25), ("Eb5", 0.25), ("F5", 0.5),
        # Bar 12
        ("G5", 0.25), ("F5", 0.25), ("Eb5", 0.5),
        # Bar 13
        ("D5", 0.25), ("C5", 0.25), ("Bb4", 0.5),
        # Bar 14
        ("F4", 0.25), ("G4", 0.25), ("Bb4", 0.5),
        # Bar 15
        ("D5", 0.25), ("Eb5", 0.25), ("F5", 0.5),
        # Bar 16
        ("G5", 0.25), ("F5", 0.25), ("Eb5", 0.5),

        # Bar 17
        ("D5", 0.5),
        # Bar 18
        ("Eb5", 0.5),
        # Bar 19
        ("F5", 0.5),
        # Bar 20
        ("G5", 1.0)
    ]

In [ ]:
tones, fs = generate_multi_octave_scale_guitar(fs=16000, duration=1.0)
melody_notes = get_demons_souls_theme_notes()  # Use the new function

melody = synthesize_melody(
    tones, fs, melody_notes,
    base_duration=3.5,  # Slow and ambient feel
    fade_ratio=0.1      # Smooth note transitions
)

# apply reverb
rir_path = "rirs/RIRS_NOISES/real_rirs_isotropic_noises/air_type1_air_binaural_stairway_1_1_90.wav"
melody = apply_rir(melody, rir_path, fs)

# Save and plot
sf.write("demons_souls_theme_synth.wav", melody, fs)


plt.figure(figsize=(10, 3))
plt.plot(melody[:10000])
plt.title("Demon's Souls Character Creation (Karplus-Strong Synth)")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Play the sound
import IPython.display as ipd
ipd.Audio("demons_souls_theme_synth.wav")

<p align="center">
  <img src="notes/Majula1.png" width="30%" />
  <img src="notes/Majula2.png" width="30%" />
  <img src="notes/Majula3.png" width="30%" />
</p>


In [ ]:
def get_majula_theme_notes():
    return [
        # Bar 1
        ("E4", 0.25), ("D4", 0.25), ("C4", 0.5),
        # Bar 2
        ("E4", 0.25), ("G4", 0.25), ("E4", 0.5),
        # Bar 3
        ("F4", 0.25), ("G4", 0.25), ("A4", 0.5),
        # Bar 4
        ("G4", 0.25), ("E4", 0.25), ("C4", 0.5),
        # Bar 5
        ("D4", 0.25), ("E4", 0.25), ("F4", 0.5),
        # Bar 6
        ("G4", 0.25), ("A4", 0.25), ("B4", 0.5),
        # Bar 7
        ("G4", 0.25), ("F4", 0.25), ("E4", 0.5),
        # Bar 8
        ("D4", 0.25), ("E4", 0.25), ("C4", 0.5),

        # Page 2 (Bar 32 onward)
        ("D4", 0.25), ("F4", 0.25), ("A4", 0.5),
        ("B4", 0.25), ("C5", 0.25), ("D5", 0.5),
        ("E5", 0.25), ("D5", 0.25), ("C5", 0.5),
        ("A4", 0.25), ("F4", 0.25), ("D4", 0.5),

        # Bar 40
        ("G4", 0.25), ("E4", 0.25), ("C4", 0.5),
        ("B3", 0.25), ("D4", 0.25), ("G4", 0.5),

        # Final cadence (Bar 96+)
        ("F4", 0.5), ("E4", 0.5),
        ("D4", 1.0), ("C4", 1.0),
        ("C4", 2.0)
    ]

In [ ]:
tones, fs =  generate_multi_octave_scale_piano(fs=16000, duration=1.0)
melody_notes = get_majula_theme_notes()

# You can adjust this to control the song speed
melody = synthesize_melody(tones, fs, melody_notes, base_duration=4, fade_ratio=0.05)

# apply reverb
rir_path = "rirs/RIRS_NOISES/real_rirs_isotropic_noises/air_type1_air_binaural_stairway_1_1_90.wav"
melody = apply_rir(melody, rir_path, fs)


# Save to WAV
sf.write("majula_synth.wav", melody, fs)

# Plot waveform
plt.figure(figsize=(10, 3))
plt.plot(melody[:10000])
plt.title("Majula (Guitar Synth)")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Play the sound
import IPython.display as ipd
ipd.Audio("majula_synth.wav")